<a href="https://colab.research.google.com/github/akashgokul/VRResearch/blob/master/Dapo_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pwd

/content/gdrive/My Drive/VRResearch


In [0]:
%cd /content/gdrive/My Drive/VRResearch

/content/gdrive/My Drive/VRResearch


In [0]:
%ls

Batch1.csv  mturk/       train_data.csv       val_data.csv
Batch2.csv  resnet18.pt  Turk_Analysis.ipynb


In [0]:
pip install ipdb

In [0]:
import torch
from PIL import Image
from torch.utils import data
import numpy as np
import ipdb
import pandas as pd
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import time
from torch import nn, optim
import torchvision.models as models


In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
class MTurkTrain(Dataset):
  def __init__(self,csv_file):
    self.data_frame = pd.read_csv(csv_file)
    self.img_dir = "/content/gdrive/My Drive/VRResearch/mturk/"

  def __len__(self):
    return self.data_frame.shape[0]
  
  def __getitem__(self,idx):
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    img_label_pair = self.data_frame.iloc[idx]
    img_name = img_label_pair[0]
    img = Image.open(self.img_dir + img_name)
    img = transform(img)
    label = img_label_pair[1]
    return img,label

params = {'batch_size': 10,
          'shuffle': True,
          'num_workers': 0}

train_dataset = MTurkTrain("train_data.csv")
training_generator = data.DataLoader(train_dataset, **params)

validation_set = MTurkTrain("val_data.csv")
validation_generator = data.DataLoader(validation_set, **params)

In [0]:

model = models.resnet18(s)
#From Stanford tutorial
#use_cuda = torch.cuda.is_available()
#device = torch.device("cuda:0" if use_cuda else "cpu")


max_epochs = 1

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters())

start_ts = time.time()
model.train()
batches = params['batch_size']

for epoch in range(max_epochs):
    print("EPOCH: " + str(epoch))
    total_loss = 0
  #Training 
    for idx, data in enumerate(training_generator):
        X, y = data[0], data[1]
        model.zero_grad()
        outputs = model(X)
        print("     on to loss")
        loss = loss_function(outputs, y)
        loss.backward()
        print("     backprop")
        optimizer.step()
        current_loss = loss.item()
        total_loss += current_loss
        print("     Loss: {:.4f}".format(total_loss/(idx+1)))
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
  


EPOCH: 0
     on to loss
     backprop
     Loss: 7.2192
     on to loss
     backprop
     Loss: 6.0114
     on to loss
     backprop
     Loss: 4.5803
     on to loss
     backprop
     Loss: 3.7484
     on to loss
     backprop
     Loss: 3.2197
     on to loss
     backprop
     Loss: 2.8144
     on to loss
     backprop
     Loss: 2.6206
     on to loss
     backprop
     Loss: 2.4007
     on to loss
     backprop
     Loss: 2.1679
     on to loss
     backprop
     Loss: 2.0566
     on to loss
     backprop
     Loss: 1.9500
     on to loss
     backprop
     Loss: 1.8973
     on to loss
     backprop
     Loss: 1.8461
     on to loss
     backprop
     Loss: 1.7966
     on to loss
     backprop
     Loss: 1.7597
     on to loss
     backprop
     Loss: 1.7005
     on to loss
     backprop
     Loss: 1.6182
     on to loss
     backprop
     Loss: 1.5402
     on to loss
     backprop
     Loss: 1.5022
     on to loss
     backprop
     Loss: 1.4715
     on to loss
     backprop
 

In [0]:
model_save_name = 'resnet18.pt'
path = F"/content/gdrive/My Drive/VRResearch/{model_save_name}"
torch.save(model.state_dict(), path)

In [0]:
model_save_name = 'resnet18.pt'
path = F"/content/gdrive/My Drive/VRResearch/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [0]:
# Validation

model.eval()
with torch.set_grad_enabled(False):
  val_wrong = 0
  for i, data in enumerate(validation_generator):
    print("Current: " + str(i) + " / " )
    # Transfer to GPU
    X, y = data[0], data[1]
     # Model computations
    outputs = model(X)
    predicted_classes = torch.max(outputs, 1)[1]
    prediction_lst = predicted_classes.tolist()
    val_wrong += sum([1 if prediction_lst[i] == y[i] else 0 for i in range(len(prediction_lst))])

print(f"Training time: {time.time()-start_ts}s")

Current: 0 / 
Current: 1 / 
Current: 2 / 
Current: 3 / 
Current: 4 / 
Current: 5 / 
Current: 6 / 
Current: 7 / 
Current: 8 / 
Current: 9 / 
Current: 10 / 
Current: 11 / 
Current: 12 / 
Current: 13 / 
Current: 14 / 
Current: 15 / 
Current: 16 / 
Current: 17 / 
Current: 18 / 
Current: 19 / 
Current: 20 / 
Current: 21 / 
Current: 22 / 
Current: 23 / 
Current: 24 / 
Current: 25 / 
Current: 26 / 
Current: 27 / 
Current: 28 / 
Current: 29 / 
Current: 30 / 
Current: 31 / 
Current: 32 / 
Current: 33 / 
Current: 34 / 
Current: 35 / 
Current: 36 / 
Current: 37 / 
Current: 38 / 
Training time: 7460.033665180206s


In [0]:
print(1 - (val_wrong / 387))

0.6408268733850129
